In [1]:
import torch

print(f"Is CUDA supported by this system? {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")

DEVICE = torch.device("cuda:0")
torch.cuda.set_device(DEVICE)
print(DEVICE)

Is CUDA supported by this system? True
CUDA version: 12.1
cuda:0


In [2]:
# !pip install transformers
# !pip install evaluate
# !pip install rouge
# !pip install spacy


import torch
import json
from tqdm import tqdm
import torch.nn as nn
from torch.optim import Adam
from nltk.translate import meteor
import nltk
import spacy
import string
import evaluate  # Bleu
from torch.utils.data import Dataset, DataLoader, RandomSampler
import pandas as pd
import numpy as np
# from pytorchtools import EarlyStopping
import transformers
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from transformers import T5Tokenizer, T5Model, T5ForConditionalGeneration, T5TokenizerFast

import warnings
warnings.filterwarnings("ignore")

2024-05-03 23:11:54.126303: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-03 23:11:54.167862: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-03 23:11:54.894372: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
def load_dataset(file_name):
    import pandas as pd
    import json
    
    df = []
    df_multi = []
    df_phrase = []
    df_passage = []
    with open('/DATA1/ai20btech11028/vojes_nlp/Data/' + file_name, encoding='utf-8') as f:
        for i in f:
            i = json.loads(i)
                
            tweet = i['postText']
            article_title = i['targetTitle']
            article = ' '.join(i['targetParagraphs'])
            spoilers = i['spoiler']
            label = i['tags']
            
            assert len(tweet) == 1
            tweet = tweet[0]
            
            assert len(label) == 1
            label = label[0]
            
            if label not in ['phrase', 'phrases', 'passage', 'multi']:
                print(label)
                
            assert label in ['phrase', 'phrases', 'passage', 'multi']
            
            if label == 'phrase' or label == 'phrases':
                label = 0
            elif label == 'multi':
                label = 2
            else:
                label = 1
                
            
            df += [{'question': tweet , 'context': i['targetParagraphs'], 'article': article_title + article, 'spoiler':spoilers,
                    'labels': label}]

    return pd.DataFrame(df)
            
    
# # test_dataset = load_dataset('test.jsonl')
train_dataset = load_dataset('train.jsonl')
validation_dataset = load_dataset('validation.jsonl')
# print(train_dataset)
train_dataset

,question,context,article,spoiler,labels
0,"Wes Welker Wanted Dinner With Tom Brady, But P...",[It’ll be just like old times this weekend for...,"Wes Welker Wanted Dinner With Tom Brady, But P...",[how about that morning we go throw?],1
1,NASA sets date for full recovery of ozone hole,[2070 is shaping up to be a great year for Mot...,Hole In Ozone Layer Expected To Make Full Reco...,[2070],0
2,This is what makes employees happy -- and it's...,"[Despite common belief, money isn't the key to...",Intellectual Stimulation Trumps Money For Empl...,[intellectual stimulation],0
3,Passion is overrated — 7 work habits you need ...,"[It’s common wisdom. Near gospel really, and n...","‘Follow your passion’ is wrong, here are 7 hab...",[Purpose connects us to something bigger and i...,2
4,The perfect way to cook rice so that it's perf...,"[Boiling rice may seem simple, but there is a ...",Revealed: The perfect way to cook rice so that...,[in a rice cooker],0
...,...,...,...,...,...
3195,Has Facebook's video explosion completely shak...,"[A long time ago in a galaxy far, far away...W...","Facebook Video Surging, But YouTube Still Offe...",[it hasn’t necessarily taken the wind out of Y...,1
3196,Cop Is Eating At A Chili's When Teen Hands Him...,"[The Kansas City, Kansas Police Department are...",Cop is eating at Chili's when teen hands him f...,"[It read, ""Thanks for keeping us safe.""]",1
3197,5 popular myths about visible signs of aging t...,"[Obama looks decades younger already, but what...",5 popular myths about visible signs of aging t...,[1. Anti-wrinkle creams will erase the fine li...,2
3198,You need to see this Twitter account that pred...,"[What the HELL?!, 1. Unless you’re somewhere w...","WTF, It Looks Like This Twitter Account ""Predi...",[@beyoncefan666],0


In [4]:
from datasets import load_dataset

dataset = load_dataset("MateuszW/generated_questions")

genQue_train = dataset['train']['generated_questions']
genQue_test = dataset['validation']['generated_questions']
# print(genQue_train)

In [5]:
train_dataset["generated_que"] = list(genQue_train)
validation_dataset["generated_que"] = list(genQue_test)
train_dataset

,question,context,article,spoiler,labels,generated_que
0,"Wes Welker Wanted Dinner With Tom Brady, But P...",[It’ll be just like old times this weekend for...,"Wes Welker Wanted Dinner With Tom Brady, But P...",[how about that morning we go throw?],1,What did Tom Brady do instead of having dinner...
1,NASA sets date for full recovery of ozone hole,[2070 is shaping up to be a great year for Mot...,Hole In Ozone Layer Expected To Make Full Reco...,[2070],0,What is the date that NASA has set for the ful...
2,This is what makes employees happy -- and it's...,"[Despite common belief, money isn't the key to...",Intellectual Stimulation Trumps Money For Empl...,[intellectual stimulation],0,What makes employees happy?
3,Passion is overrated — 7 work habits you need ...,"[It’s common wisdom. Near gospel really, and n...","‘Follow your passion’ is wrong, here are 7 hab...",[Purpose connects us to something bigger and i...,2,What are the 7 work habits that are considered...
4,The perfect way to cook rice so that it's perf...,"[Boiling rice may seem simple, but there is a ...",Revealed: The perfect way to cook rice so that...,[in a rice cooker],0,What is the best way to cook rice so that it's...
...,...,...,...,...,...,...
3195,Has Facebook's video explosion completely shak...,"[A long time ago in a galaxy far, far away...W...","Facebook Video Surging, But YouTube Still Offe...",[it hasn’t necessarily taken the wind out of Y...,1,Has Facebook's video explosion completely shak...
3196,Cop Is Eating At A Chili's When Teen Hands Him...,"[The Kansas City, Kansas Police Department are...",Cop is eating at Chili's when teen hands him f...,"[It read, ""Thanks for keeping us safe.""]",1,What did the teen give the cop?
3197,5 popular myths about visible signs of aging t...,"[Obama looks decades younger already, but what...",5 popular myths about visible signs of aging t...,[1. Anti-wrinkle creams will erase the fine li...,2,What are the 5 popular myths about visible sig...
3198,You need to see this Twitter account that pred...,"[What the HELL?!, 1. Unless you’re somewhere w...","WTF, It Looks Like This Twitter Account ""Predi...",[@beyoncefan666],0,What is the purpose of the Twitter account men...


In [6]:
TOKENIZER = T5TokenizerFast.from_pretrained('castorini/monot5-base-msmarco-10k')
MODEL = T5ForConditionalGeneration.from_pretrained('castorini/monot5-base-msmarco-10k', return_dict=True).to(DEVICE)
OPTIMIZER = Adam(MODEL.parameters(), lr=0.0001)
Q_LEN = 256   # Question Length
T_LEN = 256    # Target Length
BATCH_SIZE = 16
DEVICE = "cuda:0"
# patience = 10
# early_stopping = EarlyStopping(patience=patience, verbose=True)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [7]:
# # Loading the data

# with open('../input/squaddata/squad_train.json') as f:
#     data = json.load(f)

In [8]:
# Extracting context, question, and answers from the dataset

def prepare_data(train_dataset):
    articles = []
    
    for i in range(len(train_dataset)):
      clickbait = train_dataset.iloc[i]
      if clickbait['labels'] == 1:
        context = clickbait['article']
        question = clickbait['generated_que']
        answer = ' '.join(clickbait['spoiler'])

        inputs = {"context": context, "question": question, "answer": answer}

        articles.append(inputs)

    return articles

In [9]:
data = prepare_data(train_dataset)
# Create a Dataframe
data = pd.DataFrame(data)

In [10]:
data

,context,question,answer
0,"Wes Welker Wanted Dinner With Tom Brady, But P...",What did Tom Brady do instead of having dinner...,how about that morning we go throw?
1,Here's what happens if your Apple AirPods get ...,What is the policy of Apple regarding lost or ...,"Apple says that if AirPods are lost or stolen,..."
2,The Reason Why Gabor Kiraly Wears THOSE Tracki...,What is the reason why Gabor Kiraly wears thos...,"""The more good games I had in them, the more I..."
3,You’ll Never Believe What This Family Saw in t...,What did the family see in the sky outside the...,rainbow colours in the sky and a halo spanning...
4,Should I Drink Red Wine?5/5 say yes. Cheers to...,What are the health benefits of drinking red w...,Red wine is clearly the drink of choice if you...
...,...,...,...
1269,Women Reveal What They Think The Sexiest Part ...,What do women think is the sexiest part of a man?,almost every other part of a man is considered...
1270,Here's Another Huge Reason To Eat A Plant-Base...,What is the huge reason to eat a plant-based d...,may cut your risk for colorectal cancer -- the...
1271,If You See A Purple Butterfly Sticker At The H...,What does the purple butterfly sticker mean at...,one of her unborn twins had a condition called...
1272,"Facebook Video Surging, But YouTube Still Offe...",Has Facebook's video explosion completely shak...,it hasn’t necessarily taken the wind out of Yo...


In [11]:
class QA_Dataset(Dataset):
    def __init__(self, tokenizer, dataframe, q_len, t_len):
        self.tokenizer = tokenizer
        self.q_len = q_len
        self.t_len = t_len
        self.data = dataframe
        self.questions = self.data["question"]
        self.context = self.data["context"]
        self.answer = self.data['answer']
        
    def __len__(self):
        return len(self.questions)
    
    def __getitem__(self, idx):
        question = self.questions[idx]
        context = self.context[idx]
        answer = self.answer[idx]
        
        question_tokenized = self.tokenizer(question, context, max_length=self.q_len, padding="max_length",
                                                    truncation=True, pad_to_max_length=True, add_special_tokens=True)
        answer_tokenized = self.tokenizer(answer, max_length=self.t_len, padding="max_length", 
                                          truncation=True, pad_to_max_length=True, add_special_tokens=True)
        
        labels = torch.tensor(answer_tokenized["input_ids"], dtype=torch.long)
        labels[labels == 0] = -100
        
        return {
            "input_ids": torch.tensor(question_tokenized["input_ids"], dtype=torch.long),
            "attention_mask": torch.tensor(question_tokenized["attention_mask"], dtype=torch.long),
            "labels": labels,
            "decoder_attention_mask": torch.tensor(answer_tokenized["attention_mask"], dtype=torch.long)
        }

In [12]:
# Dataloader

train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)

train_sampler = RandomSampler(train_data.index)
val_sampler = RandomSampler(val_data.index)

qa_dataset = QA_Dataset(TOKENIZER, data, Q_LEN, T_LEN)

train_loader = DataLoader(qa_dataset, batch_size=BATCH_SIZE, sampler=train_sampler)
val_loader = DataLoader(qa_dataset, batch_size=BATCH_SIZE, sampler=val_sampler)

In [13]:
train_loss = 0
val_loss = 0
train_batch_count = 0
val_batch_count = 0

for epoch in range(20):
    MODEL.train()
    for batch in tqdm(train_loader, desc="Training batches"):
        input_ids = batch["input_ids"].to(DEVICE)
        attention_mask = batch["attention_mask"].to(DEVICE)
        labels = batch["labels"].to(DEVICE)
        decoder_attention_mask = batch["decoder_attention_mask"].to(DEVICE)

        outputs = MODEL(
                          input_ids=input_ids,
                          attention_mask=attention_mask,
                          labels=labels,
                          decoder_attention_mask=decoder_attention_mask
                        )

        OPTIMIZER.zero_grad()
        outputs.loss.backward()
        OPTIMIZER.step()
        train_loss += outputs.loss.item()
        train_batch_count += 1
    
    #Evaluation
    MODEL.eval()
    for batch in tqdm(val_loader, desc="Validation batches"):
        input_ids = batch["input_ids"].to(DEVICE)
        attention_mask = batch["attention_mask"].to(DEVICE)
        labels = batch["labels"].to(DEVICE)
        decoder_attention_mask = batch["decoder_attention_mask"].to(DEVICE)

        outputs = MODEL(
                          input_ids=input_ids,
                          attention_mask=attention_mask,
                          labels=labels,
                          decoder_attention_mask=decoder_attention_mask
                        )

        OPTIMIZER.zero_grad()
        outputs.loss.backward()
        OPTIMIZER.step()
        val_loss += outputs.loss.item()
        val_batch_count += 1

     # early_stopping(val_loss, MODEL)
        
     #    if early_stopping.early_stop:
     #        print("Early stopping")
     #        break
        
    print(f"{epoch+1}/{5} -> Train loss: {train_loss / train_batch_count}\tValidation loss: {val_loss/val_batch_count}")

Validation batches: 100%|███████████████████████████████████████████████████████████████| 16/16 [00:07<00:00,  2.08it/s]


1/5 -> Train loss: 1.7926707342267036	Validation loss: 1.144106574356556


Validation batches: 100%|███████████████████████████████████████████████████████████████| 16/16 [00:07<00:00,  2.07it/s]


2/5 -> Train loss: 1.5634514749981463	Validation loss: 0.9956340454518795


Validation batches: 100%|███████████████████████████████████████████████████████████████| 16/16 [00:08<00:00,  1.98it/s]


...

18/5 -> Train loss: 0.6286924384589333	Validation loss: 0.20145608882153787


Validation batches: 100%|███████████████████████████████████████████████████████████████| 16/16 [00:08<00:00,  1.95it/s]


19/5 -> Train loss: 0.6033218963099276	Validation loss: 0.19094626965862085


Validation batches: 100%|███████████████████████████████████████████████████████████████| 16/16 [00:08<00:00,  1.96it/s]

20/5 -> Train loss: 0.5795630504988367	Validation loss: 0.18147442810186476


In [14]:
MODEL.save_pretrained("qa_model_flant5")
TOKENIZER.save_pretrained("qa_tokenizer_flant5")

# Saved files
"""('qa_tokenizer/tokenizer_config.json',
 'qa_tokenizer/special_tokens_map.json',
 'qa_tokenizer/spiece.model',
'qa_tokenizer/added_tokens.json',
'qa_tokenizer/tokenizer.json')"""

"('qa_tokenizer/tokenizer_config.json',\n 'qa_tokenizer/special_tokens_map.json',\n 'qa_tokenizer/spiece.model',\n'qa_tokenizer/added_tokens.json',\n'qa_tokenizer/tokenizer.json')"

In [15]:
def model_performance(val_loader):
    for batch in tqdm(val_loader, desc="Validation batches"):
        input_ids = batch["input_ids"].to(DEVICE)
        attention_mask = batch["attention_mask"].to(DEVICE)
        labels = batch["labels"].to(DEVICE)
        decoder_attention_mask = batch["decoder_attention_mask"].to(DEVICE)

        # outputs = MODEL(
        #                   input_ids=input_ids,
        #                   attention_mask=attention_mask,
        #                   labels=labels,
        #                   decoder_attention_mask=decoder_attention_mask
        #                 )
            
        # # input_ids = torch.tensor(inputs["input_ids"], dtype=torch.long).to(DEVICE).unsqueeze(0)
        # # attention_mask = torch.tensor(inputs["attention_mask"], dtype=torch.long).to(DEVICE).unsqueeze(0)
    
        outputs = MODEL.generate(input_ids=input_ids, attention_mask=attention_mask)
      
        predicted_answer = TOKENIZER.decode(outputs.flatten(), skip_special_tokens=True)
    
    

In [16]:
def predict_answer(context, question, ref_answer=None):
    inputs = TOKENIZER(question, context, max_length=Q_LEN, padding="max_length", truncation=True, add_special_tokens=True)
    
    input_ids = torch.tensor(inputs["input_ids"], dtype=torch.long).to(DEVICE).unsqueeze(0)
    attention_mask = torch.tensor(inputs["attention_mask"], dtype=torch.long).to(DEVICE).unsqueeze(0)

    outputs = MODEL.generate(input_ids=input_ids, attention_mask=attention_mask)
  
    predicted_answer = TOKENIZER.decode(outputs.flatten(), skip_special_tokens=True)
    
    if ref_answer:
        # Load the Bleu metric
        bleu = evaluate.load("google_bleu")
        bleu_score = bleu.compute(predictions=[predicted_answer], 
                            references=[ref_answer])
        bert = evaluate.load("bertscore")

        bert_score = bert.compute(predictions=[predicted_answer], references=[ref_answer], lang='en')
        ref_token = nltk.word_tokenize(predicted_answer)
        pre_token = nltk.word_tokenize(ref_answer)
        # print(ref_token)
        # print(pre_token)
        meteor_score = meteor([ref_token], pre_token)
        # print("Context: \n", context)
        # print("\n")
        print("Question: ", question)
        return {
            "Reference Answer: ": ref_answer, 
            "Predicted Answer: ": predicted_answer, 
            "BLEU Score: ": bleu_score,
            "Bert Score: ": bert_score,
            "Meteor Score: ": meteor_score
        }
    else:
        return predicted_answer

In [17]:
validation_dataset

,question,context,article,spoiler,labels,generated_que
0,Five Nights at Freddy’s Sequel Delayed for Wei...,[Five Nights at Freddy’s creator Scott Cawthon...,Five Nights at Freddy’s Sequel Delayed for Wei...,[some of the plot elements are so disturbing t...,1,What is the reason for the delay of the sequel...
1,Why Arizona Sheriff Joe Arpaio’s fate could ha...,[© REUTERS/Laura Segall Maricopa County Sherif...,Why Arizona Sheriff Joe Arpaio’s fate could ha...,"[""intentionally"", could transform a court case...",2,What is the word that could determine the fate...
2,Here’s how much you should be tipping your hai...,[Here’s how much you should be tipping your ha...,Here’s how much you should be tipping your hai...,[20%],0,How much should you be tipping your hairdresser?
3,"""Harry Potter"" alums reunite for new movie",[The mythology of punk music's evolution can b...,"Alan Rickman & Rupert Grint On 'CBGB,' Reuniti...","[Alan Rickman & Rupert Grint, CBGB]",2,"What is the name of the new movie in which ""Ha..."
4,A man swallowed a microSD card and you won't b...,[PetaPixel is one of my favorite blogs. The wr...,Man swallowed a microSD card and you won't bel...,[a man who swallowed a 64GB microSD card and t...,1,What happened next?
...,...,...,...,...,...,...
795,This is what happens when you leave a hotel cl...,[Instead of encountering a mound of dirty towe...,This Is What Happens When You Leave A Hotel Cl...,[The video below shows the stunned cleaner ini...,1,What happens when you leave a hotel cleaner a ...
796,This Texas GOP elector announces that he won't...,[A Republican elector in Texas says he will no...,Texas GOP elector announces he won't vote for ...,[Christopher Suprun],0,What is the name of the Texas GOP elector who ...
797,This beauty queen cured her acne with one diet...,[Her inspirational journey is encouraging othe...,UK beauty queen cured her severe acne with one...,"[Rachel Crawley, High fat vegan plant based di...",2,What diet change cured the beauty queen's acne?
798,WikiLeaks' Julian Assange Reported Dead,"[On 16 October 2016, WikiLeaks posted a series...","WikiLeaks’ Julian Assange Isn’t Dead, Just Off...",[Julian Assange’s internet link has been inten...,1,What is the meaning of the sentence?


In [18]:
_all = {}
arr = [{} for i in range(3)]
bleu = 0
meteor_int = 0
bert = 0
count = 0

for i in range(200):
  label = validation_dataset.iloc[i]['labels']
  if label == 1:  
    question = validation_dataset.iloc[i]['generated_que']
    context = validation_dataset.iloc[i]['article']
    answer = ' '.join(validation_dataset.iloc[i]['spoiler'])
        
    print(validation_dataset.iloc[i]['labels'])
    ans = predict_answer(context, question, answer)
    _all['bleu'] = _all.get('bleu', 0) + ans["BLEU Score: "]['google_bleu']
    _all['bert'] = _all.get('bert', 0) + ans['Bert Score: ']['precision'][0]
    _all['meteor'] = _all.get('meteor', 0) + ans['Meteor Score: ']


    # type = validation_dataset.iloc[i]['labels']
    # arr[type]['bleu'] = arr[type].get('bleu',0) + ans["BLEU Score: "]['google_bleu']
    # arr[type]['bert'] = arr[type].get('bert',0) + ans["Bert Score: "]['precision'][0]
    # arr[type]['meteor'] = arr[type].get('meteor',0) + ans["Meteor Score: "]
    # arr[type]['count'] = arr[type].get('count',0) + 1
        
        
    
    print(ans)
    if validation_dataset.iloc[i]['labels'] == 2:
        continue
    count += 1
    bleu += ans["BLEU Score: "]['google_bleu']
    bert += ans["Bert Score: "]['precision'][0]
    meteor_int += ans["Meteor Score: "]
    
    

# print(" All " + str(_all['bert']/100))
# print(" Phrase " + str(arr[0]['bert']/arr[0]['count']))
# print(" Passage " + str(arr[1]['bert']/arr[1]['count']))
# print(" Multi " + str(arr[2]['bert']/arr[2]['count']))


    

1


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Question:  What is the reason for the delay of the sequel to Five Nights at Freddy's?
{'Reference Answer: ': 'some of the plot elements are so disturbing that they are making him feel sick', 'Predicted Answer: ': 'too dark', 'BLEU Score: ': {'google_bleu': 0.0}, 'Bert Score: ': {'precision': [0.8500030040740967], 'recall': [0.8265589475631714], 'f1': [0.8381170630455017], 'hashcode': 'roberta-large_L17_no-idf_version=0.3.12(hug_trans=4.41.0.dev0)'}, 'Meteor Score: ': 0.0}
1


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Question:  What happened next?
{'Reference Answer: ': 'a man who swallowed a 64GB microSD card and then pooped it into a strainer', 'Predicted Answer: ': "he couldn't puke it back up, and therefore had to poop it", 'BLEU Score: ': {'google_bleu': 0.037037037037037035}, 'Bert Score: ': {'precision': [0.8670387268066406], 'recall': [0.845383882522583], 'f1': [0.8560743927955627], 'hashcode': 'roberta-large_L17_no-idf_version=0.3.12(hug_trans=4.41.0.dev0)'}, 'Meteor Score: ': 0.18124507486209612}
1


...

Question:  What is the most common use of Tinder by college students?
{'Reference Answer: ': 'swipe left on Tinder and choose the old-fashioned way', 'Predicted Answer: ': 'college students would swipe left on Tinder and choose the old-fashioned way of finding love', 'BLEU Score: ': {'google_bleu': 0.5555555555555556}, 'Bert Score: ': {'precision': [0.9117664694786072], 'recall': [0.9358376264572144], 'f1': [0.9236452579498291], 'hashcode': 'roberta-large_L17_no-idf_version=0.3.12(hug_trans=4.41.0.dev0)'}, 'Meteor Score: ': 0.6245713305898491}
1


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Question:  What did NASA just announce?
{'Reference Answer: ': 'its next Mars rover, dubbed Mars 2020, has cleared a massive hurdle that could see it speeding toward the Red Planet in just a few years.', 'Predicted Answer: ': 'it will settle down in the soil of the Red Planet to begin its journey', 'BLEU Score: ': {'google_bleu': 0.08181818181818182}, 'Bert Score: ': {'precision': [0.8731881380081177], 'recall': [0.8443951606750488], 'f1': [0.8585503101348877], 'hashcode': 'roberta-large_L17_no-idf_version=0.3.12(hug_trans=4.41.0.dev0)'}, 'Meteor Score: ': 0.3116531165311653}
1


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Question:  What caused cars to change from being boxy in the 1980s to being curvy in the 1990
{'Reference Answer: ': 'largely due to three interrelated factors: European style trends, a government-mandated push for fuel economy, and new technologies that allowed manufacturers to more easily design and create curved shapes.', 'Predicted Answer: ': 'all of the cars look super boxy, especially compared with the curving, rounded', 'BLEU Score: ': {'google_bleu': 0.015873015873015872}, 'Bert Score: ': {'precision': [0.8462841510772705], 'recall': [0.8262279629707336], 'f1': [0.8361358642578125], 'hashcode': 'roberta-large_L17_no-idf_version=0.3.12(hug_trans=4.41.0.dev0)'}, 'Meteor Score: ': 0.0892857142857143}


In [19]:
print(bleu/count)
print(bert/count)
print(meteor_int/count)

0.14608320464180788
0.8684057418037864
0.27454697136103
